# LightWin compensation example

This notebook showcases how to compensate for a single failure.

## Imports

Note that the first import is only used to import packaged data file.
See also: https://learn.scientific-python.org/development/patterns/data-files/

In [ ]:
from importlib import resources
from pprint import pprint

# Actual mandatory imports for LightWin
from lightwin.config.config_manager import process_config
from lightwin.ui.workflow_setup import run_simulation

## Presentation of the linac

The structure of the linac is stored in a `DAT` file.
It follows the same specifications as TraceWin structure file, see [associated help page](../usage.compatibility_with_tracewin.rst) for more information.
The example can be found in `data/ads/ads.dat` (or `src/lightwin/data/ads/ads.dat` if you built LightWin from source).
Here are the first lines of the file:

In [ ]:
dat_res = resources.files("lightwin.data.ads") / "ads.dat"
dat_content = dat_res.read_text(encoding="utf-8")
print(dat_content[:892])

This is a high-power, ADS-like proton acceleractor characterized by a very high longitudinal acceptance.

## Configuration of LightWin

In order to ensure reproducibility and portability of results, all the configuration is set in a `TOML` configuration.
The full specifications are listed in [Configuration](../configuration/configuration.rst).

In [ ]:
toml_filepath = resources.files("lightwin.data.ads") / "lightwin.toml"  # Or provide Path to TOML file

Below, we associate LightWin's configuration objects with sections in the `TOML` file (the names between brackets).

In [ ]:
toml_keys = {
    "files": "files",
    "beam": "beam",
    "beam_calculator": "generic_envelope1d",
    "wtf": "generic_wtf",
    "design_space": "fit_phi_s_design_space",
    "plots": "plots_minimal",
}

This operation will check validity of the configuration, resolve paths, and  convert the `TOML` appropriate entries to a dictionary.

In [ ]:
# Adapt some configuration entries for this notebook
override = {
    "beam_calculator": {
        "flag_cython": True,  # Override flag_cython=False in [generic_envelope1d]
    },
    "wtf": {
        "k": 5,               # Override k=3 in [generic_wtf]
    },
    "plots": {
        "kwargs":
        {
            "lw": 5,          # Increase size of lines in plots
        },
    },
}
config = process_config(toml_filepath, toml_keys, override=override)

### `files` ([doc](../configuration/files.rst))

Defines the files to use, as well as where results should be saved.

In [ ]:
pprint(config["files"])

### `beam` ([doc](../configuration/beam.rst))

Define the beam properties.

In [ ]:
pprint(config["beam"])

### `beam_calculator`([doc](../configuration/beam_calculator.rst))

Define how the propagation of the beam in the linac will be calculated.
Note that you can define a `beam_calculator_post` in a similar fashion; it will be used to re-compute the propagation of the beam once the compensation settings are found.

In [ ]:
pprint(config["beam_calculator"])

### `plots` ([doc](../configuration/plots.rst))

Define the plots to create at the end of the simulation.

In [ ]:
pprint(config["plots"])

### `wtf` ([doc](../configuration/wtf.rst))

Stands for "what to fit".

In [ ]:
pprint(config["wtf"])

### `design_space` ([doc](../configuration/design_space.rst))

Defines the design space, *i.e.* the optimization variables and their bounds.

In [ ]:
pprint(config["design_space"])

## Actual usage

The following function will break and fix the linac.
Check out the source code of [the workflow_setup module](../../lightwin/lightwin.ui.workflow_setup.rst) if you want to understand and personnalize this workflow.

In [ ]:
config["plots"]["kwargs"] = {"lw": 10}

In [ ]:
fault_scenarios = run_simulation(config)

## Notes on objects hierarchy

In [ ]:
fault_scenario = fault_scenarios[0]

In [ ]:
reference_accelerator = fault_scenario.ref_acc
reference_elts = reference_accelerator.elts

In [ ]:
fixed_accelerator = fault_scenario.fix_acc
fixed_elts = fixed_accelerator.elts

In [ ]:
fixed_simulation_outputs = fixed_accelerator.simulation_outputs
pprint(fixed_simulation_outputs)

In [ ]:
solver_name = list(fixed_simulation_outputs.keys())[0]
fixed_simulation_output = fixed_simulation_outputs[solver_name]

## Accessing data

The easiest way to access the calculated data is to use the [get method](https://lightwin.readthedocs.io/en/latest/manual/get_method.html).

In [ ]:
fixed_simulation_output.get("w_kin")

In [ ]:
fixed_simulation_output.get("phi_s", to_deg=True, elt="FM53")

In [ ]:
fixed_simulation_output.get("beta", phase_space_name="phiw")

In [ ]:
fixed_simulation_output.get("envelope_energy_zdelta", elt="FM48", pos="out", to_numpy=False)

In [ ]:
fixed_accelerator.get("beta", phase_space_name="phiw")